## Word2Vec 이란?

Word2Vec 은 softmax regression 입니다. 의미 공간이라는 고차원 공간에 각 단어의 좌표값을 부여합니다.Softmax regression 은 데이터 $(X,Y)$를 이용하여, 입력된 input x가 클래스 y가 될 확률을 최대화 시키는 방향으로 클래스 y의 대표 벡터를 coefficient $\beta$ 에 학습합니다. 
<br>

$x$에 대한 $y$의 확률은 $x$와 모든 클래스 종류인 $y_{i}$와의 내적은 exponential 함수에 넣어 non-negative로 만든 뒤, 모든 expoental 함수의 합으로 나눠서 확률로 만듭니다. 
<br>

Word2vec은 window classification처럼, 긴 문장에 스캐너(window size)가 이동하며 스냅샷을 찍듯이 작동합니다. 예를들어 [a, little, cat, sit, on, the, table]이라는 4개의 단어로 cat이라는 단어를 예측하게 되는 것입니다. 4개의 단어 리스트가 $x$, 그리고 cat이 $y$가 되어 줄 것입니다. 각 단어들은 좌표값을 가지고 있습니다. $x,y$의 내적이 성립합기 위해서는 백터공간의 차원이 같아야 하겠죠? 때문에 모든 단어는 embedding 공간 속에 있는 것입니다.
<br>

Word2Vec의 학습은 의미공간에서의 각 단어의 위치좌표를 수정하는 것입니다. 각 단어의 위치좌표는 random백터로 초기화 합니다. $P(cat|[a,little,sit,on])$이 커지도록 각 단어의 좌표를 조절해야합니다. Softmax regression 은 이때, cat과 비슷하면서, 다른 단어들과의 유사도롤 낮게 만듭니다. 예를들어 cat 뿐만 아니라 dog역시 비슷한 문맥에서 등장하게 됩니다. 우리는 context vector에 Y백터가 가까워지도록 학습하고 있습니다. 때문에 목적지를 항해 위치백터를 움직이게 합니다. 

### Negative Sampling 

하지만 위 공식대로 cat을 context vector 방향으로 당겨오고, 다른 단어들을 밀어버리려면 학습량이 엄청나게 되겠죠. 때문에 cat을 제외한 몇가진 단어들만 반대로 밀어내게 됩니다. 이를 negative sampling이라고 합니다. cat은 positive sample 나머지 단어는 negative sample이 됩니다.
<br>

Negative sampling은 각 단어의 빈도를 고려해야합니다. 자주 등장한 단어를 높은 확률로 선택되도록 샘플링 확률을 만듭니다. 자주 등장한 단어만큼은 제대로 학습을 하기 위함입니다.

## Doc2vec

Documents 는 word들의 연속이며, 하나의 상위 위계라고 해석가능하다. 이제 의미 공간에 document의 위치좌표를 학습하는 방법이 제안됩니다. Doc2vec은 documents i를 하나의 단어처럼 생각합니다. 그리고 모든 스냅샷에서 다른 단어들의 위치좌표와 함께 평균을 취하여 context vector 를 만듭니다. 그 다음은 Word2Vec 과 같습니다. document id + 4 개의 단어로 이뤄진 context vector 에 가깝도록 Y, cat 의 위치를 조절합니다.

![one-hot encodings](assets/doc2vec.png)

In [2]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [3]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [4]:
vector = model.infer_vector(["system", "response"])

## Doc2Vec Class

> __class gensim.models.doc2vec.Doc2Vec__(documents=None, corpus_file=None, dm_mean=None, dm=1, dbow_words=0, dm_concat=0, dm_tag_count=1, docvecs=None, docvecs_mapfile=None, comment=None, trim_rule=None, callbacks=(), **kwargs)
Bases: gensim.models.base_any2vec.BaseWordEmbeddingsModel

학습을 시키는 Class로 문장과 문서의 분산된 표상을 설명하는 Neural networks입니다.

#### wv
> Word2VecKeyedVectors

이 object는 단어와 임베딩 사이의 맵핑관계를 토함하고 있습니다. 학습 이후에, 이것은 쿼리를 임베딩 공간으로 보내줍니다.

#### docvecs
> Doc2VecKeyedVectors 

이 object 는 paragraph vectors를 포함하고 있습니다. 워드 백터와 함께 문단 백터도 함꼐 임베딩 되어있다는 사실입니다.

#### vocabulary
> Doc2VecVocab 

이것은 vocabulary를 호출시킵니다 (gensim의 Dictionary와 비슷하다). 모든 unique한 단어들을 찾는 것이 아니라, extra functionality를 보여줍니다. 마치 단어 빈도의 sorting 이나, 극단적으로 rare한 단어에 관하여 보여줍니다.

skip-gram과 함께하는 PV-DBOW("dm=0, dbow_words=1") 모드를 본다면, 워드와 도큐먼트 백터는 기본적으로 같은 공간속에 존재합니다. 왜냐하면 그들은 predictive neural-network를 같은 공간속에서 제공받습니다. 이것은,  어떠한 개별적인 차원의 magnitudes(word & doc vector)가 같은 forward-propagation과 back-propagation의 영향을 받는다는 소리입니다.